# Johnson-Mehl tessellation picture

I'd like to draw a picture of the Johnson-Mehl tessellation and a Voronoi tessellation so the viewer can compare them.

One interesting fact which I hadn't realised until trying to draw these pictures: the boundaries between cells in the JM tessellation _aren't straight_.

I've used the algorithm described by Moulinec in ["A simple and fast algorithm for computing discrete Voronoi, Johnson-Mehl or Laguerre diagrams of points"](https://www.sciencedirect.com/science/article/pii/S0965997822000618). Mainly because it's simple, although being fast is also an advantage.

## Outline
1. Sample the arrival times and locations, and prune them (i.e. delete seeds which arrived in a covered region)
2. Assign each pixel in an image to its appropriate cell, based on which Johnson-Mehl seed covers it first.
3. Draw the picture with boundaries.

In [ ]:
import numpy as np
from unconstrained import sample_points, prune_arrivals
from tqdm import trange, tqdm
from PIL import Image, ImageDraw
from datetime import datetime

### Sampling arrival times and pruning

In [ ]:
def sample_disc_points(sample_size):
    # Chooses points uniformly in disc centered at (1/2,1/2) with radius 1/2.
    theta = 2*np.pi*np.random.random(size=(sample_size,1))
    radius_sqrt = np.sqrt(np.random.random(size=(sample_size,1)))
    x = 0.45*radius_sqrt * np.cos(theta) + 0.5
    y = 0.45*radius_sqrt * np.sin(theta) + 0.5
    return np.concatenate((x,y),axis=1)

def get_arrival_times( rho, max_time=1.0, R=0 ):
    rate = rho*(1+2*R)**2
    Nmax = int(max_time*rate + 2*np.sqrt(max_time*rate)) # Two standard deviations above the mean
    interarrival_times = np.random.exponential(scale=1/rate,size=Nmax)
    arrival_times = np.cumsum(interarrival_times)
    too_late = np.searchsorted(arrival_times,max_time,side='right') # First index where the arrival time is at least max_time
    while too_late == Nmax: # This will be the case if we are unlucky and Nmax points arrived before time max_time. We'll just generate more points.
        interarrival_times = np.append(interarrival_times, np.random.exponential(scale=1/rate,size=Nmax))
        arrival_times = np.cumsum(interarrival_times)
        too_late = np.searchsorted(arrival_times,max_time,side='right') # First index where the arrival time is at least max_time
    return arrival_times[:too_late].copy()

In [ ]:
rate = 80 # `rate` is the rate at which points arrive in the disc, rather than the arrival rate per unit area, rho. It corresponds to rho * |A| in the paper.
# Since A is a disc of radius 0.45, `rate` is rho * pi * 0.45 * 0.45.
max_time = 2*( (2*np.log(rate) + 4*np.log(np.log(rate))) / (np.pi*rate) )**(1/3)

times = get_arrival_times(rate,max_time=max_time)
seeds = sample_disc_points(len(times))
arrived = prune_arrivals(times, seeds)
print(f'{len(arrived)} out of {len(times)} seeds germinated.')
times = times[arrived]
seeds = seeds[arrived]
times = times - np.min(times)

### Assigning pixels to their cells

This is the bit using Moulinec's method. Moulinec has two separate steps: first assigning the pixels which are covered by time $T$, then the pixels which were not covered by time $T$. He chooses $T$ to optimise the speed of the algorithm. We can simplify the algorithm by choosing $T$ to be at least the coverage time, then there is no second step. That's a little bit slower than choosing the optimal $T$, but since we have a decent upper bound on $T$ it's not too bad.

---

The algorithm works as follows: we start with an array $\mathcal{D}$ of "running minimum coverage times" and an array $\mathcal{I}$ of assignments, both the same shape as the output image. We intialise $\mathcal{D}$ to be full of $\infty$. We order the seeds $x_1, \dots, x_N$ with corresponding arrival times $t_1, \dots, t_N$.

Then for each $i = 1, \dots, N$ in turn: for every pixel $y$ in the ball centred at $x_i$ of radius $T-t_i$, this pixel was first reached by seed $i$ at time $\| x_i - y \| + t_i$. If $\| x_i - y \| + t_i < \mathcal{D}(y)$, then we set $\mathcal{I}(y) = i$ (overwriting its previous value if it had one) and set the new running minimum $\mathcal{D}(y) = \| x_i - y \| + t_i$.

Once we have done this for all $N$ seeds, every pixel is correctly assigned.

In [ ]:
def get_ball_pixels(centre, radius, img_size):
    """
    Returns the indices of the pixels in the picture
    corresponding to a ball of a given radius
    centred at a given point.
    Also saves the corresponding (squared) distances.
    
    I suspect a numpy-ish method would be faster:
    create a 2d array containing the (squared) distance between each point in [min_i,max_i]x[min_j,max_j]
    and v, then turn that into an array of bools which we can return along with the distances.
    We might need to also then return (min_i, min_j) so the bool array can be aligned within the image.    
    """
    if radius <= 0:
        return [], []
    v = (img_size-1)*centre
    x,y = v[0], v[1]
    r = (img_size-1)*radius
    r2 = r*r
    min_i = max( 0, int(x-r) )
    max_i = min( img_size-1, int(x+r)+1 )
    min_j = max( 0, int(y-r) )
    max_j = min( img_size-1, int(y+r)+1 )
    in_ball = []
    sq_distances = []
    for i in range(min_i, max_i+1):
        dx2 = (x-i)*(x-i)
        if dx2 > r2:
            continue
        w = np.sqrt( r2 - dx2 )
        for j in range(max(int(y-w),min_j), min(int(y+w)+2,max_j+1)):
            d2 = dx2 + (y-j)**2
            if d2 <= r2:
                in_ball.append((i,j))
                sq_distances.append(d2)
    return in_ball, sq_distances

def assign_cells( seeds, times, img_size, T=1.0 ):
    """
    Assigns all the pixels in an img_size x img_size picture
    to their respective Johnson-Mehl cells.
    T should be a decent upper bound on the coverage time - smaller T
    means we check fewer points.
    This is a modified version of Moulinec's algorithm,
    in which we assign things which were covered by time T,
    and leave the rest unassigned.
    """
    min_cov_times = np.full((img_size,img_size),np.inf) # running minimum coverage times
    assignments = np.full((img_size,img_size),-1,dtype=int) # everything uncovered is assigned to a separate class.

    for i in trange(len(times)):
        xi = seeds[i]
        ti = times[i]
        indices, d2s = get_ball_pixels(xi, T-ti, img_size)
        for k, ij_pair in enumerate(indices):
            cov_time = np.sqrt(d2s[k])/img_size + ti
            if cov_time < min_cov_times[ij_pair]:
                assignments[ij_pair] = i
                min_cov_times[ij_pair] = cov_time
    return assignments, min_cov_times

In [ ]:
img_size = 2000

# Compute the assignment and coverage time of each pixel,
# and the location of the last point covered.

I_full, cov_times = assign_cells(seeds, times, img_size, T=max_time)
# print(I)
A,_ = get_ball_pixels(np.array([0.5,0.5]), 0.45, img_size)
I = np.full((img_size,img_size),-1,dtype=int)
max_cov_time = 0
max_time_location = (0,0)
for ij in A:
    I[ij] = I_full[ij]
    if cov_times[ij] > max_cov_time:
        max_cov_time = cov_times[ij]
        max_time_location = ij

### Finding the boundaries

We'll go through each pixel in the image and check if it is near more than one cell.
If it is then we declare it part of a boundary and colour it black.
This is a very inefficient way to do it, but it's not the slowest bit of the algorithm.
Thinner boundaries are computed faster.

In [ ]:
boundary_thickness = 0.0015
# Go through I and if any point has more than one cell nearby to it, colour it black.
in_boundary = np.full((img_size,img_size),0,dtype=int)
spacing = np.linspace(0,1,num=img_size,endpoint=True)
for i, x in enumerate(tqdm(spacing)):
    for j, y in enumerate(spacing):
        v = np.array([x,y])
        local_ball,_ = get_ball_pixels(v,boundary_thickness,img_size)
        cells = { I[ij] for ij in local_ball }
        if len(cells) >= 2:
            in_boundary[i,j] = 1

In [ ]:
# The earliest points should be bright,
# the latest points pale.
def seed_colour(i):
    # Returns the colour of the ith seed.
    tmax = np.max(times)
    return (255, int(190*times[i]/tmax), int(190*times[i]/tmax))

In [ ]:
cell_colour = (255,255,255)
boundary_colour = (0,0,0)
seed_size = 0.01*img_size
box_size = 0.03*img_size

data = np.full((img_size, img_size, 3),255, dtype=np.uint8)
image = Image.fromarray(data)
draw = ImageDraw.Draw(image)
for i,v in enumerate(seeds):
    x,y = (img_size-1)*v
    draw.ellipse([y-0.5*seed_size,x-0.5*seed_size,
                  y+0.5*seed_size,x+0.5*seed_size],
                 fill=seed_colour(i))
N = I.shape[0]
for i in range(N):
    for j in range(N):
        if (i - 0.5*(img_size-1))**2 + (j - 0.5*(img_size-1))**2 > (0.45*(img_size))**2:
            draw.point([j,i],fill=cell_colour)
        if in_boundary[i,j]:
            draw.point([j,i],fill=boundary_colour)

draw.rectangle([max_time_location[1]-0.5*box_size,max_time_location[0]-0.5*box_size,
                max_time_location[1]+0.5*box_size,max_time_location[0]+0.5*box_size],outline="blue",width=10)

# image.show() # opens in system image viewer
display(image)

In [ ]:
# image.show()
now = datetime.now().isoformat()
image.save(f"jm-pictures/jm-tessellation-{now}.png")

## The whole thing in a loop
This is essentially all the above code copied into one loop,
if you want to generate a large number of images.
We did this to make sure it was visible to the naked eye
that all vertices in the tessellation have degree 3.
In some samples two vertices can be so close together it looks a bit like
there is a single vertex of degree $> 3$.
The minimum spacing of vertices in a JM tessellation (and Poisson-Voronoi tessellation)
seems like an interesting problem to look at.

In [ ]:
while True:
    rho = 80
    max_time = 2*( (2*np.log(rho) + 4*np.log(np.log(rho))) / (np.pi*rho) )**(1/3)

    times = get_arrival_times(rho,max_time=max_time)
    seeds = sample_disc_points(len(times))
    arrived = prune_arrivals(times, seeds)
    # print(f'{len(arrived)} out of {len(times)} seeds germinated.')
    times = times[arrived]
    seeds = seeds[arrived]
    times = times - np.min(times)
    
    img_size = 2000
    
    # Compute the assignment and coverage time of each pixel,
    # and the location of the last point covered.
    
    I_full, cov_times = assign_cells(seeds, times, img_size, T=max_time)
    # print(I)
    A,_ = get_ball_pixels(np.array([0.5,0.5]), 0.45, img_size)
    I = np.full((img_size,img_size),-1,dtype=int)
    max_cov_time = 0
    max_time_location = (0,0)
    for ij in A:
        I[ij] = I_full[ij]
        if cov_times[ij] > max_cov_time:
            max_cov_time = cov_times[ij]
            max_time_location = ij
    # ## Uncomment this bit if you want to condition on the last covered point being in the interior.
    # ## In the limit that has a positive probability, but about 4%, so expect to discard a lot of samples. 
    # if (max_time_location[0] - 0.5*(img_size-1))**2 + (max_time_location[1] - 0.5*(img_size-1))**2 > (0.44*img_size)**2:
    #     print("Last covered point is on the boundary, discarding sample.")
    #     continue
    
    boundary_thickness = 0.0015
    # Go through I and if any point has more than one cell nearby to it, colour it black.
    in_boundary = np.full((img_size,img_size),0,dtype=int)
    spacing = np.linspace(0,1,num=img_size,endpoint=True)
    for i, x in enumerate(tqdm(spacing)):
        for j, y in enumerate(spacing):
            v = np.array([x,y])
            local_ball,_ = get_ball_pixels(v,boundary_thickness,img_size)
            cells = { I[ij] for ij in local_ball }
            if len(cells) >= 2:
                in_boundary[i,j] = 1
    
    # The earliest points should be bright,
    # the latest points pale.
    def seed_colour(i):
        # Returns the colour of the ith seed.
        tmax = np.max(times)
        return (255, int(190*times[i]/tmax), int(190*times[i]/tmax))
    
    cell_colour = (255,255,255)
    boundary_colour = (0,0,0)
    seed_size = 0.01*img_size
    box_size = 0.03*img_size
    
    data = np.full((img_size, img_size, 3),255, dtype=np.uint8)
    image = Image.fromarray(data)
    draw = ImageDraw.Draw(image)
    for i,v in enumerate(seeds):
        x,y = (img_size-1)*v
        draw.ellipse([y-0.5*seed_size,x-0.5*seed_size,
                      y+0.5*seed_size,x+0.5*seed_size],
                     fill=seed_colour(i))
    N = I.shape[0]
    for i in range(N):
        for j in range(N):
            if (i - 0.5*(img_size-1))**2 + (j - 0.5*(img_size-1))**2 > (0.45*(img_size))**2:
                draw.point([j,i],fill=cell_colour)
            if in_boundary[i,j]:
                draw.point([j,i],fill=boundary_colour)
    
    draw.rectangle([max_time_location[1]-0.5*box_size,max_time_location[0]-0.5*box_size,
                    max_time_location[1]+0.5*box_size,max_time_location[0]+0.5*box_size],outline="blue",width=10)

    # image.show()
    now = datetime.now().isoformat()
    image.save(f"jm-pictures/jm-tessellation-{now}.png")